In [1]:
import requests

1. Get class data

In [19]:
import requests

cookies = {
    'csrftoken': '7uoFMfLL1lLpZ9raoHvGJ1BrzWE8udxhLANeRSfEMa4XasPTt5Y3MNBd3vyX9nXH',
    'sessionid': '23ivwpc2mtrobpkjy4tdzjsr49wq5mqw',
}

headers = {
    'authority': 'penncoursereview.com',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8',
    'accept-language': 'en-US,en;q=0.9',
    # 'cookie': 'csrftoken=7uoFMfLL1lLpZ9raoHvGJ1BrzWE8udxhLANeRSfEMa4XasPTt5Y3MNBd3vyX9nXH; sessionid=23ivwpc2mtrobpkjy4tdzjsr49wq5mqw',
    'sec-ch-ua': '"Chromium";v="116", "Not)A;Brand";v="24", "Brave";v="116"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"macOS"',
    'sec-fetch-dest': 'document',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-site': 'cross-site',
    'sec-fetch-user': '?1',
    'sec-gpc': '1',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36',
}

response = requests.get('https://penncoursereview.com/api/review/course/CIS-1210?format=json&token=platform', cookies=cookies, headers=headers)
print(response.text)

{"code":"CIS-1210","name":"Prog Lang and Tech II","description":"This is a course about Algorithms and Data Structures using the JAVA programming language. We introduce the basic concepts about complexity of an algorithm and methods on how to compute the running time of algorithms. Then, we describe data structures like stacks, queues, maps, trees, and graphs, and we construct efficient algorithms based on these representations. The course builds upon existing implementations of basic data structures in JAVA and extends them for the structures like trees, studying the performance of operations on such structures, and theiefficiency when used in real-world applications. A large project introducing students to the challenges of software engineering concludes the course.","aliases":[],"historical_codes":[{"full_code":"CIS-121","branched_from":false,"semester":"2022A"}],"latest_semester":"2023C","num_sections":39,"num_sections_recent":1,"instructors":{"130":{"id":130,"average_reviews":{"rA